In [30]:
"""This section creates the ini files for VRT."""
import os

project_path = 'D:/ERCOT/AEP Dynamic/MQT/'
project_list = os.listdir(project_path)

vrt_path = 'D:/DMView 2.7.1/'

for project in project_list:
    initials = ['lag','lead']
    for initial in initials:
        """Create the ini file."""
        filename = vrt_path + project + '_VRT_' + initial + '.ini'

        """Write the ini file."""
        with open(filename, 'w') as ini:
            ini.write('/' + project + 'for ERCOT dynamic model quality test\n')
            ini.write("/created by Arthur Li - CF Power\n\n")
            ini.write("[Build FS]\nBuild_FS_flag = 1\nPOI_PF = ")
            if initial == 'lag':
                ini.write('0.95 /lagging \n')
            else:
                ini.write('-0.95 /leading \n')

            ini.write('\n[Input files]\ninput_path = CASEs\\'+project+'\n')
            ini.write('unconv_casefile ='+project+'.sav /psse v33\n model_file_lst = [\''+project+'.dyr\']/codes in v33\n')
            ini.write('\n[Tests]\nTest01_LVRT_LEGACY = [\'VOLT\',\'DATAs\\ERCOT_Legacy_LVRT.xlsx\']\n')
            ini.write('Test02_LVRT_VOLT_DIP = [\'VOLT\',\'DATAs\\ERCOT-245_2800_Piecewise_LVRT_(PV-BESS)_10secSpacing_r2.xlsx\']\n')
            ini.write('Test3_HVRT_PREFERED = [\'VOLT\',\'DATAs\\ERCOT-245_2800_HVRT.xlsx\']\n\n[Settings]\nPlot_Flag = 1')

In [31]:
os.chdir(vrt_path)

for project in project_list:
    for initial in initials:
        filename = vrt_path + project + '_VRT_' + initial + '.ini'
        os.system('py -2 dmview.pyc "'+filename+'"')
    print('Finish the VRT of '+project+' successfully.\n')


Finish the VRT of 24INR0201 & 27INR0029 Short Creek Solar & Storage successfully.

Finish the VRT of 24INR0378 Solara BESS 1 successfully.

Finish the VRT of 24INR0379 Solara BESS 2 successfully.

Finish the VRT of 24INR0442 Midnight Sun Energy Storage successfully.

